<div style="background-color: #8b0000; padding: 10px; text-align: center;">
  <h1 style="color: #ffffff; text-shadow: 0 0 10px #ff6666;">Next Word Prdictor Using LSTM</h1>
</div>


<div style="background-color: #f0f0f0; padding: 20px; border-radius: 10px;">
  <h3 style="color: #333333; text-align: center; font-weight: bold;">What Is Next Word Prediction?</h3>
  <p style="color: #666666; text-align: justify;">Next word prediction is a natural language processing (NLP) task that aims to predict the most likely word or sequence of words that will follow a given input text. Iat's a fundamental problem in NLP and has various applications, including text generation, autocomplete suggestions, and machine translation.</p>
  <div style="background-color: #4CAF50; color: #ffffff; padding: 10px; border-radius: 5px; text-align: center;">
    <p style="margin: 0; text-align: center;">In This Notebook I Will Use Deep LSTM Model For Solve This Problem</p>
  </div>
</div>


<div style="background-color: #8b0000; padding: 10px; text-align: center;">
  <h1 style="color: #ffffff; text-shadow: 0 0 10px #ff6666;">Import Important Modules</h1>
</div>


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional,Input,Attention,LayerNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

2024-10-08 18:34:37.584165: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 18:34:37.652708: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 18:34:37.672370: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 18:34:37.802680: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-08 18:34:38.720126: W tensorflow/compiler/tf2

<div style="background-color: #8b0000; padding: 10px; text-align: center;">
  <h1 style="color: #ffffff; text-shadow: 0 0 10px #ff6666;">Data Preparation</h1>
</div>


In [2]:
medium_df = pd.read_csv('medium_data.csv')
news_df = pd.read_csv('ArticlesMarch2018.csv')
large_df = pd.read_csv('train.csv')

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


I0000 00:00:1728399880.163670     573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728399880.296831     573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728399880.296914     573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
large_df.shape

(120000, 3)

In [5]:
large_df = large_df.drop_duplicates()

In [6]:

large_df.head(1)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."


In [7]:
large_df.shape

(120000, 3)

In [8]:
large_df = large_df.head(15000)

In [9]:
medium_df.head(1)

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30


In [10]:
medium_df['title'][1]

'Hands-on Graph Neural Networks with PyTorch & PyTorch Geometric'

In [11]:
news_df.head(1)

,articleID,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL,articleWordCount
0,5a974697410cf7000162e8a4,By BINYAMIN APPELBAUM,article,"Virtual Coins, Real Resources","['Bitcoin (Currency)', 'Electric Light and Pow...",1,Business,1,01-03-2018 00:17,Economy,America has a productivity problem. One explan...,The New York Times,News,https://www.nytimes.com/2018/02/28/business/ec...,1207


In [12]:
news_df['headline'][1]

'U.S. Advances Military Plans for North Korea'

In [13]:
print(medium_df.shape)
print(news_df.shape)

(6508, 10)
(1385, 15)


In [14]:
# Creating New DataFrame
text = pd.concat([medium_df['title'], news_df['headline'],large_df['Title']], ignore_index=True)

In [15]:
data = pd.DataFrame({
    'text' : text
})

In [16]:
data['text'][8]

'Which 2020 Candidate is the Best at\xa0Twitter?'

In [17]:
data.isnull().sum()

text    0
dtype: int64

In [18]:
data.duplicated().sum()

886

In [19]:
data.shape

(22893, 1)

In [20]:
data.drop_duplicates(inplace=True)

In [21]:
data.shape

(22007, 1)

<div style="background-color: #8b0000; padding: 10px; text-align: center;">
  <h1 style="color: #ffffff; text-shadow: 0 0 10px #ff6666;">Data Cleaning</h1>
</div>


In [22]:
data['text'] = data['text'].apply(lambda x: x.replace(u'\xa0',u' '))
data['text'] = data['text'].apply(lambda x: x.replace('\u200a', ' '))


In [23]:
import re
import html

def clean_text(text):
    # Unescape HTML escape characters
    cleaned_text = html.unescape(text)
    # Remove HTML tags
    cleaned_text = re.sub(r'<[^>]+>', '', cleaned_text)
    return cleaned_text

data['text'] = data['text'].apply(clean_text)


In [24]:
print(data['text'][15000])

Oil steady in edgy Asia trading


In [25]:
len(data)

22007

In [27]:
data = data[:10000]

<div style="background-color: #8b0000; padding: 10px; text-align: center;">
  <h1 style="color: #ffffff; text-shadow: 0 0 10px #ff6666;">Tokenization</h1>
</div>



  <div style="background-color: #4CAF50; color: #ffffff; padding: 10px; border-radius: 5px; text-align: center;">
    <p style="margin: 0; text-align: center;">Tokenizatin is the process which we provide an unique id to all the words and make word index or we can say vocabulary</p>
  </div>
</div>


In [28]:
tokenizer = Tokenizer(oov_token='<oov>')
tokenizer.fit_on_texts(data['text'])
total_words = len(tokenizer.word_index) + 1


print('Total number of words: ', total_words)
print('Word: ID')
print("--------------")
print("<oov>: ", tokenizer.word_index['<oov>'])
print('Storng: ', tokenizer.word_index['strong'])
print('And: ', tokenizer.word_index['and'])
print('Consumption: ', tokenizer.word_index['consumption'])

Total number of words:  12551
Word: ID
--------------
<oov>:  1
Storng:  721
And:  7
Consumption:  9495


<div style="background-color: #f0f0f0; padding: 20px; border-radius: 10px;">
  <h3 style="color: #333333; text-align: center; font-weight: bold;">Title Text Into Sequences</h3>
  <p style="color: #666666; text-align: justify;">let we have sentence like "My Name Is Hakim" and this will convert into sequeces with their respective tokens{"I': 1, "Name": 2, "Is": 3, "Hakim": 4. So output will be [1,2,3,4]. Likewise all of our inputs will be like this</p>
  <div style="background-color: #4CAF50; color: #ffffff; padding: 10px; border-radius: 5px; text-align: center;">
    <p style="margin: 0; text-align: center;">Let's Make Sequences</p>
  </div>
</div>


In [29]:
from tqdm.notebook import tqdm
input_sequences = []
for line in tqdm(data['text']):
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


  0%|          | 0/10000 [00:00<?, ?it/s]

<div style="background-color: #f0f0f0; padding: 20px; border-radius: 10px;">
  <h3 style="color: #333333; text-align: center; font-weight: bold;">Make all Titles in Same Length</h3>
  <p style="color: #666666; text-align: justify; text-align: center;">The length of every title has to be in same length.To make it we need to find the title which has maximum length.</p>
  <div style="background-color: #4CAF50; color: #ffffff; padding: 10px; border-radius: 5px; text-align: center;">
    <p style="margin: 0; text-align: center;">Making Pad Sequences</p>
  </div>
</div>


In [30]:
max_sequence_len = max(len(x) for x in input_sequences)
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

<div style="background-color: #8b0000; padding: 10px; text-align: center;">
  <h1 style="color: #ffffff; text-shadow: 0 0 10px #ff6666;">Prepare Features and Labels</h1>
</div>


<div style="background-color: #f0f0f0; padding: 20px; border-radius: 10px;">
  <p style="color: #666666; text-align: justify; text-align: center;">Here, we consider last element of all sequences as a label. Then we need to perform encoding on labels corresponding to tatall_words</p>

In [31]:
# create features and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [32]:
print(xs.shape)
print(ys.shape)

(67154, 21)
(67154, 12551)


In [33]:
# Modeling
model = Sequential()
model.add(Embedding(total_words, 100, input_length= max_sequence_len - 1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.summary()

/home/hope1300/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1728399897.660128     573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728399897.660221     573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728399897.660249     573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728399897.822508     573 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728399897.822589    

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [34]:
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
history = model.fit(xs, ys, epochs=300, verbose=1)

In [ ]:
model.save('final_model.h5')

<div style="background-color: #8b0000; padding: 10px; text-align: center;">
  <h1 style="color: #ffffff; text-shadow: 0 0 10px #ff6666;">Predicting Next Word Of Title</h1>
</div>


In [ ]:
seed_text = 'i am '
next_words = 3
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted_probabilities = model.predict(token_list, verbose=0)  # Use predict() instead of predict_classes()
    predicted_index = np.argmax(predicted_probabilities)  # Get the index of the word with highest probability
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
            
    seed_text += " " + output_word
print(seed_text)


In [ ]:
import pickle
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

    
with open('max_sequence_length', 'wb') as file:
    pickle.dump(max_sequence_len, file)
    
model.save('text_predictor.h5')

In [ ]:
# Function to generate recommendations based on user input
def generate_recommendations(seed_text, next_words=6):
    # Use trained model to predict next words
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted_probabilities = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probabilities)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Capture user input
user_input = input("Enter your query: ")

# Generate and display recommendations
recommendations = generate_recommendations(user_input)
print("Recommendations:", recommendations)
